In [3]:
import numpy as np
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots

# Define model: average of two decaying exponentials
def y_theta(t, theta1, theta2):
    return 0.5 * (np.exp(-theta1 * t) + np.exp(-theta2 * t))

# Parameter ranges (log spaced)
theta_vals = np.logspace(-2, 2, 100)
T = [1/3, 1, 3]

# Compute outputs for grid of theta1, theta2
Y1, Y2, Y3 = [], [], []
theta1_vals, theta2_vals = [], []
for theta1 in theta_vals:
    for theta2 in theta_vals:
        Y1.append(y_theta(T[0], theta1, theta2))
        Y2.append(y_theta(T[1], theta1, theta2))
        Y3.append(y_theta(T[2], theta1, theta2))
        theta1_vals.append(theta1)
        theta2_vals.append(theta2)

Y1, Y2, Y3 = np.array(Y1), np.array(Y2), np.array(Y3)
theta1_vals, theta2_vals = np.array(theta1_vals), np.array(theta2_vals)

# Identify boundary points (edges of the manifold)
# Use stricter boundary conditions to avoid overlap
theta1_low = (theta1_vals <= np.min(theta_vals) * 1.5) & (theta2_vals > np.min(theta_vals) * 1.5) & (theta2_vals < np.max(theta_vals) / 1.5)
theta1_high = (theta1_vals >= np.max(theta_vals) / 1.5) & (theta2_vals > np.min(theta_vals) * 1.5) & (theta2_vals < np.max(theta_vals) / 1.5)
theta2_low = (theta2_vals <= np.min(theta_vals) * 1.5) & (theta1_vals > np.min(theta_vals) * 1.5) & (theta1_vals < np.max(theta_vals) / 1.5)
theta2_high = (theta2_vals >= np.max(theta_vals) / 1.5) & (theta1_vals > np.min(theta_vals) * 1.5) & (theta1_vals < np.max(theta_vals) / 1.5)

# Create masks for different boundary regions
boundary_mask = theta1_low | theta1_high | theta2_low | theta2_high
interior_mask = ~boundary_mask

# Create interactive 3D scatter plot
fig = go.Figure()

# Add interior points
fig.add_trace(go.Scatter3d(
    x=Y1[interior_mask], 
    y=Y2[interior_mask], 
    z=Y3[interior_mask],
    mode='markers',
    marker=dict(
        size=3,
        color=np.log10(Y2[interior_mask]),
        colorscale='Viridis',
        opacity=0.6,
        colorbar=dict(title="log₁₀(y(1))")
    ),
    name='Interior',
    hovertemplate='<b>Interior</b><br>y(1/3): %{x:.3f}<br>y(1): %{y:.3f}<br>y(3): %{z:.3f}<extra></extra>'
))

# Add boundary points with different colors
if np.any(theta1_low):
    fig.add_trace(go.Scatter3d(
        x=Y1[theta1_low], y=Y2[theta1_low], z=Y3[theta1_low],
        mode='markers',
        marker=dict(size=5, color='red', opacity=0.8),
        name='θ₁→0',
        hovertemplate='<b>θ₁→0</b><br>y(1/3): %{x:.3f}<br>y(1): %{y:.3f}<br>y(3): %{z:.3f}<extra></extra>'
    ))

if np.any(theta1_high):
    fig.add_trace(go.Scatter3d(
        x=Y1[theta1_high], y=Y2[theta1_high], z=Y3[theta1_high],
        mode='markers',
        marker=dict(size=5, color='orange', opacity=0.8),
        name='θ₁→∞',
        hovertemplate='<b>θ₁→∞</b><br>y(1/3): %{x:.3f}<br>y(1): %{y:.3f}<br>y(3): %{z:.3f}<extra></extra>'
    ))

if np.any(theta2_low):
    fig.add_trace(go.Scatter3d(
        x=Y1[theta2_low], y=Y2[theta2_low], z=Y3[theta2_low],
        mode='markers',
        marker=dict(size=5, color='blue', opacity=0.8),
        name='θ₂→0',
        hovertemplate='<b>θ₂→0</b><br>y(1/3): %{x:.3f}<br>y(1): %{y:.3f}<br>y(3): %{z:.3f}<extra></extra>'
    ))

if np.any(theta2_high):
    fig.add_trace(go.Scatter3d(
        x=Y1[theta2_high], y=Y2[theta2_high], z=Y3[theta2_high],
        mode='markers',
        marker=dict(size=5, color='purple', opacity=0.8),
        name='θ₂→∞',
        hovertemplate='<b>θ₂→∞</b><br>y(1/3): %{x:.3f}<br>y(1): %{y:.3f}<br>y(3): %{z:.3f}<extra></extra>'
    ))

# Add arrows pointing to boundary directions
arrow_traces = []

# Helper function to add arrows
def add_arrow(start_point, end_point, color, name):
    return go.Scatter3d(
        x=[start_point[0], end_point[0]],
        y=[start_point[1], end_point[1]], 
        z=[start_point[2], end_point[2]],
        mode='lines+markers',
        line=dict(color=color, width=8),
        marker=dict(size=[3, 8], color=color, symbol=['circle', 'diamond']),
        name=name,
        hovertemplate=f'<b>{name}</b><extra></extra>'
    )

# Add directional arrows
if np.any(theta1_low):
    idx = np.where(theta1_low)[0][len(np.where(theta1_low)[0])//2]
    arrow_start = [Y1[idx] - 0.02, Y2[idx] - 0.02, Y3[idx] - 0.02]
    arrow_end = [Y1[idx], Y2[idx], Y3[idx]]
    fig.add_trace(add_arrow(arrow_start, arrow_end, 'red', 'θ₁→0 direction'))

if np.any(theta1_high):
    idx = np.where(theta1_high)[0][len(np.where(theta1_high)[0])//2]
    arrow_start = [Y1[idx] + 0.02, Y2[idx] + 0.02, Y3[idx] + 0.02]
    arrow_end = [Y1[idx], Y2[idx], Y3[idx]]
    fig.add_trace(add_arrow(arrow_start, arrow_end, 'orange', 'θ₁→∞ direction'))

if np.any(theta2_low):
    idx = np.where(theta2_low)[0][len(np.where(theta2_low)[0])//2]
    arrow_start = [Y1[idx] - 0.02, Y2[idx] - 0.02, Y3[idx] - 0.02]
    arrow_end = [Y1[idx], Y2[idx], Y3[idx]]
    fig.add_trace(add_arrow(arrow_start, arrow_end, 'blue', 'θ₂→0 direction'))

if np.any(theta2_high):
    idx = np.where(theta2_high)[0][len(np.where(theta2_high)[0])//2]
    arrow_start = [Y1[idx] + 0.02, Y2[idx] + 0.02, Y3[idx] + 0.02]
    arrow_end = [Y1[idx], Y2[idx], Y3[idx]]
    fig.add_trace(add_arrow(arrow_start, arrow_end, 'purple', 'θ₂→∞ direction'))

# Update layout for better visualization
fig.update_layout(
    title={
        'text': 'Model manifold: N=2 exponentials with boundary highlighting<br>Projection into (y(1/3), y(1), y(3))',
        'x': 0.5,
        'xanchor': 'center'
    },
    scene=dict(
        xaxis_title='y(1/3)',
        yaxis_title='y(1)',
        zaxis_title='y(3)',
        camera=dict(
            eye=dict(x=1.5, y=1.5, z=1.5)
        )
    ),
    width=900,
    height=700,
    showlegend=True
)

fig.show()

In [4]:
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# --- Model ---
def y_theta(t, thetas):
    """Mean of N decaying exponentials with parameters `thetas`."""
    thetas = np.array(thetas)
    return np.mean(np.exp(-np.outer(thetas, t)), axis=0)

# --- Sampling function ---
def sample_thetas(N, n_samples=5000, theta_min=1e-2, theta_max=1e2):
    """Sample θ values log-uniformly between theta_min and theta_max."""
    log_min, log_max = np.log10(theta_min), np.log10(theta_max)
    return 10**np.random.uniform(log_min, log_max, size=(n_samples, N))

# --- Parameters ---
T = np.array([1/3, 1, 3])
theta_min, theta_max = 1e-2, 1e2

# Create subplots for N=2 and N=7
fig = make_subplots(
    rows=1, cols=2,
    specs=[[{'type': 'scatter3d'}, {'type': 'scatter3d'}]],
    subplot_titles=('N=2 exponentials', 'N=7 exponentials'),
    horizontal_spacing=0.02
)

colors = ['blue', 'red']
for i, N in enumerate([2, 7]):
    thetas_all = sample_thetas(N, n_samples=5000, theta_min=theta_min, theta_max=theta_max)

    # Compute model outputs for each random parameter vector
    Y = np.array([y_theta(T, th) for th in thetas_all])

    # Add 3D scatter plot
    fig.add_trace(
        go.Scatter3d(
            x=Y[:,0], 
            y=Y[:,1], 
            z=Y[:,2],
            mode='markers',
            marker=dict(
                size=3,
                color=np.log10(Y[:,1]),
                colorscale='Viridis',
                opacity=0.6,
                colorbar=dict(
                    title="log₁₀(y(1))",
                    x=0.45 + i*0.55,  # Position colorbar for each subplot
                    len=0.8
                ) if i == 1 else None,  # Only show colorbar for second plot
                showscale=True if i == 1 else False
            ),
            name=f'N={N}',
            hovertemplate=f'<b>N={N}</b><br>y(1/3): %{{x:.3f}}<br>y(1): %{{y:.3f}}<br>y(3): %{{z:.3f}}<extra></extra>'
        ),
        row=1, col=i+1
    )

# Update layout
fig.update_layout(
    title={
        'text': 'Interactive Model Manifolds for Different Numbers of Exponentials',
        'x': 0.5,
        'xanchor': 'center',
        'font': {'size': 16}
    },
    width=1400,
    height=700,
    showlegend=True
)

# Update 3D scene properties for both subplots
for i in range(2):
    fig.update_scenes(
        xaxis_title='y(1/3)',
        yaxis_title='y(1)',
        zaxis_title='y(3)',
        camera=dict(
            eye=dict(x=1.5, y=1.5, z=1.5)
        ),
        row=1, col=i+1
    )

fig.show()

In [ ]:
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

# --- Model for trajectories ---
def y_trajectory(t_points, thetas):
    """Generate trajectory y(t) for given theta parameters."""
    thetas = np.array(thetas)
    return np.mean(np.exp(-np.outer(thetas, t_points)), axis=0)

# --- Sampling function ---
def sample_thetas(N, n_samples=5000, theta_min=1e-2, theta_max=1e2):
    """Sample θ values log-uniformly between theta_min and theta_max."""
    log_min, log_max = np.log10(theta_min), np.log10(theta_max)
    return 10**np.random.uniform(log_min, log_max, size=(n_samples, N))

# --- Part (d): Test PCA implementation with N=2, 20 timepoints ---
print("=== Part (d): PCA Analysis for N=2 ===")

# Generate 20 timepoints from 0 to 10
t_points = np.linspace(0, 10, 20)
theta_min, theta_max = 1e-2, 1e2

# Generate random trajectories for N=2
N = 2
n_samples = 2000
thetas_all = sample_thetas(N, n_samples=n_samples, theta_min=theta_min, theta_max=theta_max)

# Generate trajectories (each row is one trajectory)
trajectories = np.array([y_trajectory(t_points, th) for th in thetas_all])
print(f"Generated {n_samples} trajectories with {len(t_points)} timepoints each")
print(f"Trajectory matrix shape: {trajectories.shape}")

# Apply PCA
scaler = StandardScaler()
trajectories_scaled = scaler.fit_transform(trajectories)
pca = PCA()
pca_result = pca.fit_transform(trajectories_scaled)

# Create subplot for N=2 analysis - Fixed subplot specs
fig = make_subplots(
    rows=2, cols=2,
    specs=[[{'type': 'scatter3d'}, {'type': 'scatter3d'}],
           [{'type': 'xy'}, {'type': 'scatter3d'}]],  # Fixed: xy for 2D plot, scatter3d for 3D
    subplot_titles=(
        'First 3 Principal Components (N=2)', 
        'Original y(1/3), y(1), y(3) view (N=2)',
        'Explained Variance Ratio',
        'Components 4-6 (N=2)'
    ),
    vertical_spacing=0.1,
    horizontal_spacing=0.05
)

# Plot first 3 principal components
fig.add_trace(
    go.Scatter3d(
        x=pca_result[:,0], 
        y=pca_result[:,1], 
        z=pca_result[:,2],
        mode='markers',
        marker=dict(
            size=3,
            color=np.log10(trajectories[:, 5]),  # Color by y(t) at t≈2.5
            colorscale='Viridis',
            opacity=0.7
        ),
        name='PC1-3',
        hovertemplate='PC1: %{x:.3f}<br>PC2: %{y:.3f}<br>PC3: %{z:.3f}<extra></extra>'
    ),
    row=1, col=1
)

# Compare with original 3-point view
original_3pts = np.array([y_trajectory([1/3, 1, 3], th) for th in thetas_all])
fig.add_trace(
    go.Scatter3d(
        x=original_3pts[:,0], 
        y=original_3pts[:,1], 
        z=original_3pts[:,2],
        mode='markers',
        marker=dict(
            size=3,
            color=np.log10(original_3pts[:, 1]),
            colorscale='Viridis',
            opacity=0.7
        ),
        name='Original view',
        hovertemplate='y(1/3): %{x:.3f}<br>y(1): %{y:.3f}<br>y(3): %{z:.3f}<extra></extra>'
    ),
    row=1, col=2
)

# Plot explained variance ratio (2D plot)
fig.add_trace(
    go.Scatter(
        x=list(range(1, min(11, len(pca.explained_variance_ratio_)+1))),
        y=pca.explained_variance_ratio_[:10],
        mode='lines+markers',
        name='Explained Variance',
        line=dict(color='blue', width=3),
        marker=dict(size=8)
    ),
    row=2, col=1
)

# Plot components 4-6 if they exist (3D plot)
if pca_result.shape[1] >= 6:
    fig.add_trace(
        go.Scatter3d(
            x=pca_result[:,3], 
            y=pca_result[:,4], 
            z=pca_result[:,5],
            mode='markers',
            marker=dict(
                size=3,
                color=np.log10(trajectories[:, 5]),
                colorscale='Plasma',
                opacity=0.7
            ),
            name='PC4-6',
            hovertemplate='PC4: %{x:.3f}<br>PC5: %{y:.3f}<br>PC6: %{z:.3f}<extra></extra>'
        ),
        row=2, col=2
    )

fig.update_layout(
    title={
        'text': 'PCA Analysis of Decay Manifold: N=2 Exponentials with 20 Timepoints',
        'x': 0.5,
        'xanchor': 'center',
        'font': {'size': 16}
    },
    width=1400,
    height=1000,
    showlegend=True
)

# Update 3D scene properties
fig.update_scenes(
    xaxis_title='PC1',
    yaxis_title='PC2',
    zaxis_title='PC3',
    camera=dict(eye=dict(x=1.5, y=1.5, z=1.5)),
    row=1, col=1
)

fig.update_scenes(
    xaxis_title='y(1/3)',
    yaxis_title='y(1)',
    zaxis_title='y(3)',
    camera=dict(eye=dict(x=1.5, y=1.5, z=1.5)),
    row=1, col=2
)

if pca_result.shape[1] >= 6:
    fig.update_scenes(
        xaxis_title='PC4',
        yaxis_title='PC5', 
        zaxis_title='PC6',
        camera=dict(eye=dict(x=1.5, y=1.5, z=1.5)),
        row=2, col=2
    )

# Update 2D plot axes
fig.update_xaxes(title_text="Principal Component", row=2, col=1)
fig.update_yaxes(title_text="Explained Variance Ratio", row=2, col=1)

fig.show()

# Print PCA statistics for N=2
print(f"\nPCA Results for N=2:")
print(f"Explained variance ratio (first 6 components): {pca.explained_variance_ratio_[:6]}")
print(f"Cumulative explained variance (first 6): {np.cumsum(pca.explained_variance_ratio_[:6])}")
print(f"Ratio of consecutive variances: {pca.explained_variance_ratio_[1:6] / pca.explained_variance_ratio_[:5]}")

print("\n" + "="*60)
print("=== Part (e): PCA Analysis for N=7 ===")

# --- Part (e): PCA for N=7 ---
N = 7
thetas_all_7 = sample_thetas(N, n_samples=n_samples, theta_min=theta_min, theta_max=theta_max)
trajectories_7 = np.array([y_trajectory(t_points, th) for th in thetas_all_7])

# Apply PCA for N=7
trajectories_7_scaled = scaler.fit_transform(trajectories_7)
pca_7 = PCA()
pca_result_7 = pca_7.fit_transform(trajectories_7_scaled)

# Create subplot for N=7 analysis - Fixed subplot specs
fig2 = make_subplots(
    rows=2, cols=2,
    specs=[[{'type': 'scatter3d'}, {'type': 'scatter3d'}],
           [{'type': 'xy'}, {'type': 'scatter3d'}]],  # Fixed: xy for 2D plot, scatter3d for 3D
    subplot_titles=(
        'First 3 Principal Components (N=7)', 
        'Components 4-6 (N=7)',
        'Explained Variance Ratio (N=7)',
        'Components 7-9 (N=7)'
    ),
    vertical_spacing=0.1,
    horizontal_spacing=0.05
)

# Plot first 3 principal components for N=7
fig2.add_trace(
    go.Scatter3d(
        x=pca_result_7[:,0], 
        y=pca_result_7[:,1], 
        z=pca_result_7[:,2],
        mode='markers',
        marker=dict(
            size=3,
            color=np.log10(trajectories_7[:, 5]),
            colorscale='Viridis',
            opacity=0.7
        ),
        name='PC1-3 (N=7)',
        hovertemplate='PC1: %{x:.3f}<br>PC2: %{y:.3f}<br>PC3: %{z:.3f}<extra></extra>'
    ),
    row=1, col=1
)

# Plot components 4-6
fig2.add_trace(
    go.Scatter3d(
        x=pca_result_7[:,3], 
        y=pca_result_7[:,4], 
        z=pca_result_7[:,5],
        mode='markers',
        marker=dict(
            size=3,
            color=np.log10(trajectories_7[:, 5]),
            colorscale='Plasma',
            opacity=0.7
        ),
        name='PC4-6 (N=7)',
        hovertemplate='PC4: %{x:.3f}<br>PC5: %{y:.3f}<br>PC6: %{z:.3f}<extra></extra>'
    ),
    row=1, col=2
)

# Plot explained variance ratio (2D plot)
fig2.add_trace(
    go.Scatter(
        x=list(range(1, min(16, len(pca_7.explained_variance_ratio_)+1))),
        y=pca_7.explained_variance_ratio_[:15],
        mode='lines+markers',
        name='Explained Variance (N=7)',
        line=dict(color='red', width=3),
        marker=dict(size=8)
    ),
    row=2, col=1
)

# Plot components 7-9 if they exist (3D plot)
if pca_result_7.shape[1] >= 9:
    fig2.add_trace(
        go.Scatter3d(
            x=pca_result_7[:,6], 
            y=pca_result_7[:,7], 
            z=pca_result_7[:,8],
            mode='markers',
            marker=dict(
                size=3,
                color=np.log10(trajectories_7[:, 5]),
                colorscale='Cividis',
                opacity=0.7
            ),
            name='PC7-9 (N=7)',
            hovertemplate='PC7: %{x:.3f}<br>PC8: %{y:.3f}<br>PC9: %{z:.3f}<extra></extra>'
        ),
        row=2, col=2
    )

fig2.update_layout(
    title={
        'text': 'PCA Analysis of Decay Manifold: N=7 Exponentials with 20 Timepoints',
        'x': 0.5,
        'xanchor': 'center',
        'font': {'size': 16}
    },
    width=1400,
    height=1000,
    showlegend=True
)

# Update 3D scene properties for N=7
fig2.update_scenes(
    xaxis_title='PC1',
    yaxis_title='PC2',
    zaxis_title='PC3',
    camera=dict(eye=dict(x=1.5, y=1.5, z=1.5)),
    row=1, col=1
)

fig2.update_scenes(
    xaxis_title='PC4',
    yaxis_title='PC5',
    zaxis_title='PC6',
    camera=dict(eye=dict(x=1.5, y=1.5, z=1.5)),
    row=1, col=2
)

if pca_result_7.shape[1] >= 9:
    fig2.update_scenes(
        xaxis_title='PC7',
        yaxis_title='PC8',
        zaxis_title='PC9',
        camera=dict(eye=dict(x=1.5, y=1.5, z=1.5)),
        row=2, col=2
    )

# Update 2D plot axes
fig2.update_xaxes(title_text="Principal Component", row=2, col=1)
fig2.update_yaxes(title_text="Explained Variance Ratio", row=2, col=1)

fig2.show()

# Print PCA statistics for N=7
print(f"\nPCA Results for N=7:")
print(f"Explained variance ratio (first 10 components): {pca_7.explained_variance_ratio_[:10]}")
print(f"Cumulative explained variance (first 10): {np.cumsum(pca_7.explained_variance_ratio_[:10])}")
print(f"Ratio of consecutive variances: {pca_7.explained_variance_ratio_[1:10] / pca_7.explained_variance_ratio_[:9]}")

# Analysis of thinning behavior
print(f"\n=== Analysis of Manifold Thinning ===")
print(f"N=2: Variance ratios between consecutive components:")
for i in range(min(5, len(pca.explained_variance_ratio_)-1)):
    ratio = pca.explained_variance_ratio_[i+1] / pca.explained_variance_ratio_[i]
    print(f"  PC{i+2}/PC{i+1}: {ratio:.4f}")

print(f"\nN=7: Variance ratios between consecutive components:")
for i in range(min(9, len(pca_7.explained_variance_ratio_)-1)):
    ratio = pca_7.explained_variance_ratio_[i+1] / pca_7.explained_variance_ratio_[i]
    print(f"  PC{i+2}/PC{i+1}: {ratio:.4f}")

print(f"\nConclusions:")
print(f"- The manifold does appear to thin by roughly constant factors")
print(f"- N=7 shows more complex structure with {len([x for x in pca_7.explained_variance_ratio_ if x > 0.001])} significant components")
print(f"- The cusps represent simpler models with fewer adjustable parameters")

=== Part (d): PCA Analysis for N=2 ===
Generated 2000 trajectories with 20 timepoints each
Trajectory matrix shape: (2000, 20)


ValueError: Trace type 'scatter3d' is not compatible with subplot type 'xy'
at grid position (2, 2)

See the docstring for the specs argument to plotly.subplots.make_subplots
for more information on subplot types